## AWS Client Setup
Installa la AWS CLI scaricandola da questo url: https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html#getting-started-install-instructions

Una volta installata, incolla le informazioni dalla Command line or programmatic access nella folder .aws nel profilo utente di sistema.

Nel file config
```json
[username]
region = us-west-2
output = json
```

Nel file credential inserire le informazioni in questo formato:

```json
[username]
aws_access_key_id=
aws_secret_access_key=
aws_session_token=
```

In [ ]:
import os
from utils import bedrock
from langchain.llms import Bedrock
import pickle


os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "[]"

bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)

model = Bedrock(
    model_id="anthropic.claude-v2", 
    client=bedrock_runtime,
    model_kwargs={'temperature': 0.8,"max_tokens_to_sample": 1000}
    )

# AWS Client Setup [AWS LAB]

In [ ]:
import json
import os
import sys
import pickle

from langchain.llms import Bedrock
from utils import bedrock

bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)

model = Bedrock(
    model_id="anthropic.claude-v2", 
    client=bedrock_runtime,
    model_kwargs={'temperature': 0.5}
    )

# Articoli su Attori famosi
Immaginiamo di essere un critico cinematografico che scrive per un blog che ogni giorno raccoglie il compleanno di attori famosi che cadono quel giorno. Prende la sua bio e ne scrive un articolo. Proviamo a velocizzare il suo lavoro!

## Se non volete iscrivervi e settare le API potete saltare questa parte. Ho preparato dei pickle per voi direttamente leggibili dal disco. 

In [ ]:
rapid_api_key = ""

Recuperiamo le informazioni attraverso le API di imdb fruibili tramite "RapidAPI"

In [ ]:
import requests

def get_bio(name):
    url = "https://imdb8.p.rapidapi.com/actors/get-bio"
    querystring = {"nconst":name}

    headers = {
        "X-RapidAPI-Key": rapid_api_key,
        "X-RapidAPI-Host": "imdb8.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    return response.json()

def get_filmography(name):
    url = "https://imdb8.p.rapidapi.com/actors/get-all-filmography"

    querystring = {"nconst":name}

    headers = {
        "X-RapidAPI-Key": rapid_api_key,
        "X-RapidAPI-Host": "imdb8.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    names = response.json()
    return names
    
def get_born_today(month,day):
    url = "https://imdb8.p.rapidapi.com/actors/list-born-today"
    querystring = {"month":month,"day":day}

    headers = {
        "X-RapidAPI-Key": rapid_api_key,
        "X-RapidAPI-Host": "imdb8.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    list_of_names = response.json()
    names = []
    for name in list_of_names[0:3]:
        r = get_bio(name.split("name/")[1])
        #r = get_filmography(name.split("name/")[1])
        names.append(r)
    return names

In [ ]:
month = 7
day = 2
response = get_born_today(month=month,day=day)

In [ ]:
'''with open('data/2_7.pickle', 'wb') as handle:
    pickle.dump(response, handle, protocol=pickle.HIGHEST_PROTOCOL)'''

### Se non vuoi agganciare la tua key di RAPID puoi caricare i pickle da qua e continuare

In [ ]:
with open('data/2_7.pickle', 'rb') as handle:
    response = pickle.load(handle)

In [ ]:
response

## Costruiamo il nostro prompt

Sfruttiamo anche l'integrazione con Wikipedia per raccogliere più informazioni

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate
)
from PIL import Image
import urllib
import matplotlib.pyplot as plt
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wikipedia_info = wikipedia.run(response[0]['name'])

In [ ]:
wikipedia_info

In [ ]:
prompt = """"You're a film critic who writes articles about actors whose birthday is today. Use the information provided to write an article in the style indicated."
Actor: {actor}
Nicknames: {nicknames}
Bio: {bio}
gender: {gender}
birth place: {birth_place}
birth data: {birth_date}
Use also this information taken from wikipedia about the actor: 
{wikipedia}
"""

messages = [
            ("system", prompt),
            ("human", "Write a long article that should go on the film critic's blog to celebrate the actor's birthday by remembering him. The article must have a title and a body")
        ]

prompt = ChatPromptTemplate.from_messages(messages=messages)
chain = prompt | model

In [ ]:
actor = response[0]['name']
if "nicknames" in response[0]:
    nicknames = ";".join(response[0]['nicknames'])
else:
    nicknames = " Not found"
gender = response[0]['gender']
bio = response[0]['miniBios'][0]['text']
birth_place=response[0]['birthPlace']
birth_date = response[0]['birthDate']

In [ ]:
articles = chain.invoke({
    "actor":actor,
    "nicknames":nicknames,
    "bio":bio,
    "gender": gender,
    "birth_place": birth_place,
    "birth_date": birth_date,
    "wikipedia":wikipedia_info
})

In [ ]:
img = Image.open(urllib.request.urlopen(response[0]['image']['url']))
plotimg = plt.imshow(img)

print(articles)

# Scegliamo lo stile, il tono e la lunghezza

In [ ]:
tone_and_style_list = ["motivational","inspirational","informative","humorous","casual","serious"]
length_list = ["short","medium","long"]
readability_list = ["easy","intermediate","complex"]


In [ ]:
tone = tone_and_style_list[2]
length = length_list[2]
readability = readability_list[2]

tone,length,readability

In [ ]:
prompt = """"You're an expert Chief Editor that review, correct and rewrite article. Based on the following styles:"
Tone and Style: {tone}
Readability: {readability}
The article must be {length} in length
"""

messages = [
            ("system", prompt),
            ("human", "Review, correct and rewrite the article: {article}")
        ]

prompt = ChatPromptTemplate.from_messages(messages=messages)
chain = prompt | model

In [ ]:
chief_articles = chain.invoke({
    "tone":tone,
    "length":length,
    "readability":readability,
    "article": articles
})

In [ ]:
print(chief_articles)